In [3]:
import pandas as pd
import re # regular expressions
from datasets import Dataset

c:\Users\gaova\anaconda3\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset

In [4]:
data_path = '../data/naruto.csv'
transcriptionOfNaruto = pd.read_csv(data_path)

In [5]:
transcriptionOfNaruto.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [6]:
#Remove actions from transcript
def remove_actions(text):
    return re.sub(r'\(.*?\)', '', text)

In [7]:
transcriptionOfNaruto['line'] = transcriptionOfNaruto['line'].apply(remove_actions)

In [8]:
transcriptionOfNaruto.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [9]:
transcriptionOfNaruto['number_of_words']=transcriptionOfNaruto['line'].str.strip().str.split(" ") # splitting the lines into words
transcriptionOfNaruto['number_of_words']=transcriptionOfNaruto['number_of_words'].apply(lambda x: len(x)) #counting the number of words in each line

In [10]:
transcriptionOfNaruto.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [11]:
transcriptionOfNaruto['naruto_response_flag']=0
transcriptionOfNaruto.loc[(transcriptionOfNaruto['name']=='Naruto') & (transcriptionOfNaruto['number_of_words']>5),'naruto_response_flag']=1

In [12]:
transcriptionOfNaruto

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,27,0
159,Iruka,Huh?,1,0
160,Naruto,Iruka Sensei!,2,0
161,Iruka,Ah! That hurts!,3,0


In [13]:
index_to_take = list(transcriptionOfNaruto[(transcriptionOfNaruto['naruto_response_flag']==1)&(transcriptionOfNaruto.index>0)].index)

In [14]:
index_to_take[:3]

[6, 28, 30]

In [15]:
system_promt = """ You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pat
tern. You can use the following responses as a reference:
1. "I'm gonna be the next Hokage!"
2. "I'm gonna do it!"
3. "I'm gonna go for it!" """


prompts = []
for i in index_to_take:
    prompt = system_promt

    prompt += transcriptionOfNaruto.iloc[i-1]['line']
    prompt += '\n'
    prompt += transcriptionOfNaruto.iloc[i]['line']
    prompts.append(prompt)

In [16]:
print(prompts[3])

 You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pat
tern. You can use the following responses as a reference:
1. "I'm gonna be the next Hokage!"
2. "I'm gonna do it!"
3. "I'm gonna go for it!"  Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? 
  Now that’s some serious motivation! I’ll have this clean in no time!


In [17]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,"You are Naruto from the anime ""Naruto"". Your ..."
1,"You are Naruto from the anime ""Naruto"". Your ..."
2,"You are Naruto from the anime ""Naruto"". Your ..."
3,"You are Naruto from the anime ""Naruto"". Your ..."
4,"You are Naruto from the anime ""Naruto"". Your ..."


In [18]:
dataset = Dataset.from_pandas(df)